In [ ]:
import sys
home_dir = '/Users/an88/'
sys.path.insert(0, home_dir + 'Dropbox/Documents (Academic)/GitHub/NUTS (Python)/')
sys.path.insert(0, home_dir + 'Dropbox/Documents (Academic)/GitHub/Discontinuous HMC/')
sys.path.insert(0, home_dir + 'Dropbox/Documents (Academic)/GitHub/Discontinuous HMC/dhmc/')
from importlib import reload
from dhmc_sampler import DHMCSampler
import NUTS
from mcmc_diagnostic import *

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle as pkl
import warnings
import pdb
import time
%matplotlib inline

### Import Jolly-Seber model with black-kneed capsid data from Seber (1982)

In [ ]:
from jolly_seber_model import *

#### Test the gradient function.

In [ ]:
phi0 = .8 * np.ones(len(index_phi))
p0 = .15 * np.ones(len(index_p))
U_init0 = 500
B0 = 200 * np.ones(len(index_U) - 1)
theta0 = pack_param(p0, phi0, U_init0, B0)

#### Test the coordinate wise update function

In [ ]:
scale = np.ones(n_param)
dhmc = DHMCSampler(f, f_update, n_disc, n_param, scale)
dhmc.test_cont_grad(theta0, sd=1, n_test=10);
_, theta, logp_fdiff, logp_diff = \
    dhmc.test_update(theta0, sd=10, n_test=10)

### Define an integrator for discontinuous HMC

In [ ]:
def check_output(samples, logp_samples, accept_prob, nfevals_per_itr):
    
    filename = 'jolly_seber_test_output.py'
    with open(filename, 'rb') as file:
        samples0, logp_samples0, accept_prob0, nfevals_per_itr0 \
            = pkl.load(file)
            
    test_pass = np.allclose(samples, samples0) \
        and np.allclose(logp_samples, logp_samples0) \
        and np.allclose(accept_prob, accept_prob0) \
        and np.allclose(nfevals_per_itr, nfevals_per_itr0)
        
    if test_pass:
        print('Test passed! The current output matches the former one.')

In [ ]:
seed = 1
n_burnin = 10 ** 1
n_sample = 1 * 10 ** 1
n_update = 1
dt = .025 * np.array([.8, 1])
nstep = [70, 85]

samples, logp_samples, accept_prob, nfevals_per_itr, time_elapsed = \
    dhmc.run_sampler(theta0, dt, nstep, n_burnin, n_sample, 
                     seed=seed, n_update=n_update)
samples = samples[n_burnin:, :]
check_output(samples, logp_samples, accept_prob, nfevals_per_itr)

In [ ]:
update_test_output = False
if update_test_output:
    filename = 'jolly_seber_test_output.py'
    with open(filename, 'wb') as file:
        to_save = (samples, logp_samples, accept_prob, nfevals_per_itr)
        pkl.dump(to_save, file)

In [ ]:
mcmc_output = {
    'samples': samples,
    'logp': logp_samples,
    'accept_prob': accept_prob,
    'nfevals_per_itr': nfevals_per_itr,
    'n_burnin': n_burnin,
    'seed': seed,
    'theta0': theta0,
}

In [ ]:
filename = 'jolly_seber_dhmc_output.npy'
with open(filename, 'wb') as file:
    pkl.dump(mcmc_output, file)

### NUTS-Gibbs sampler for comparison.

In [ ]:
def update_U(theta):
    
    # Extract each parameter.
    tilphi = theta[index_phi]
    tilp = theta[index_p]
    phi = 1 / (1 + np.exp(-tilphi))
    p = 1 / (1 + np.exp(-tilp))
    log_U = theta[index_U]
    U = np.floor(np.exp(log_U)).astype('int64') 
    
    for index in range(len(U)):
        U[index] = cond_update_U(phi, p, U, index)
    
    theta[index_U] = np.log(U)
    return theta

def cond_update_U(phi, p, U, index):
    
    # The possible values of U[index].
    U_i = np.arange(u[index], n_max)
    
    # Log-likelihood of the 1st capture.
    logp = \
        log_factorial[U_i] - log_factorial[U_i - u[index]] + U_i * np.log(1 - p[index]) 
    
    # Contributions from prior. Take advantage of Markovian structure.    
    if index == 0:
        logp += - np.log(U_i)
    else:
        U_prev_var = sigma_B ** 2 + \
            phi[index - 1] * (1 - phi[index - 1]) * (U - u)[index - 1]
        logp += - np.log(U_prev_var) / 2 + \
            - (U_i - phi[index - 1] * (U[index - 1] - u[index - 1])) ** 2 / U_prev_var / 2
    
    if index < len(U) - 1:
        U_i_var = sigma_B ** 2 + \
            phi[index] * (1 - phi[index]) * (U_i - u[index])
        logp += - np.log(U_i_var) / 2 + \
            - (U[index + 1] - phi[index] * (U_i - u[index])) ** 2 / U_i_var / 2
    
    # Multi-nomial sampling
    prob = np.exp(logp - np.max(logp))
    prob = prob / np.sum(prob)
    
    return np.random.choice(U_i, size=1, p=prob)

#### The integrator for NUTS.

In [ ]:
def leapfrog(f, epsilon, theta0, p0, grad0):

    p = p0.copy()
    theta = theta0.copy()
    grad = grad0.copy()

    p = p + 0.5 * epsilon * grad
    theta = theta + epsilon * p
    logp, grad = f(theta)
    p = p + 0.5 * epsilon * grad
    nfevals = 1

    return theta, p, grad, logp, nfevals

NUTS.integrator = leapfrog
NUTS.compute_hamiltonian = lambda logp, p: logp - 0.5 * np.dot(p, p)
NUTS.random_momentum = lambda d: np.random.randn(d)

In [ ]:
%%prun
n_burnin = 10 ** 3
n_mcmc = 1 * 10 ** 3
n_update = 10
seed = 1

# Run Gibbs with NUTS update for continuous variables
theta = theta0.copy()
n_per_update = math.ceil(n_mcmc / n_update)
nfevals_total = 0
samples = np.zeros((n_mcmc + n_burnin, len(theta)))
logp_samples = np.zeros(n_mcmc + n_burnin)
accept_prob = np.zeros(n_mcmc + n_burnin)

np.random.seed(seed)
def f_cond(theta_cont):
    logp, grad, _ = f(np.concatenate((theta_cont, theta[n_cont:])))
    if not np.any(np.isnan(grad)):
        grad = grad[:n_cont]
    return logp, grad
logp, grad = f_cond(theta)
for i in range(n_mcmc + n_burnin):
    def f_cond(theta_cont):
        logp, grad, _ = f(np.concatenate((theta_cont, theta[n_cont:])))
        if not np.any(np.isnan(grad)):
            grad = grad[:n_cont]
        return logp, grad
#     theta_cont, logp, grad, accept_prob[i], nfevals = \
#         NUTS.HMC(f_cond, np.random.uniform(2 * 10 ** -2, 2.5 * 10 ** -2), np.random.randint(50, 60), theta[:n_cont], logp, grad)
    theta_cont, logp, grad, accept_prob[i], nfevals = \
        NUTS.NUTS(f_cond, np.random.uniform(2 * 10 ** -2, 2.5 * 10 ** -2), theta[:n_cont], logp, grad, max_depth=8)
    theta[:n_cont] = theta_cont
    theta = update_U(theta)
    nfevals_total += nfevals + 1
    samples[i, :] = theta
    logp_samples[i] = logp
    if (i + 1) % n_per_update == 0:
        print('{:d} iterations have been completed.'.format(i+1))
        
nfevals_per_itr = nfevals_total / (n_mcmc + n_burnin)
print('Each iteration required {:.2f} likelihood evaluations on average.'.format(nfevals_per_itr))
samples = samples[n_burnin:, :]
gibbs_samples = samples.copy()

In [ ]:
seed = None
mcmc_output = {
    'samples': samples,
    'logp': logp_samples,
    'accept_prob': accept_prob,
    'nfevals_per_itr': nfevals_per_itr,
    'n_burnin': n_burnin,
    'seed': seed,
    'theta0': theta0,
}

In [ ]:
filename = 'jolly_seber_gibbs_output.npy'
with open(filename, 'wb') as file:
    pkl.dump(mcmc_output, file)

### Try M-H sampler with an optimal proposal variance.

In [ ]:
def RWMH_step(f, theta0, logp0, stepsize, Sigma=None):
    """
    Params
    ------
    f : function
        Computes the log density of the target density
    stepsize : scalar or vector
        Proposal std is scaled by 'diag(stepsize).'
    Sigma : matrix
        Covariance matrix. If given, the proposal variance
        will be 'stepsize ** 2 * Sigma.'
    """
    theta = theta0.copy()
    if Sigma is None:
        theta += stepsize * np.random.randn(len(theta0))
    else:
        theta += stepsize * np.random.multivariate_normal(np.zeros(len(theta)), Sigma)
    logp = f(theta)
    accept_prob = min(1, math.exp(logp - logp0))
    accepted = accept_prob > np.random.uniform()
    if not accepted:
        theta = theta0
        logp = logp0
        
    return theta, logp, accepted, accept_prob

def dual_average(mcmc_step, chain_state, stepsize, accept_target, n_iter):
    """
    Adjusts the stepsize of the proposal kernel to achieve the target acceptance rate.
    
    Params
    ------
    mcmc_step : function(chain_state, stepsize)
        Takes a dict of the current state and output a dict of the next
        state. The dict must have a key 'accept_prob'.
    stepsize : scalar
        Initial stepsize to try.
        
    Returns
    -------
    stepsize_bar : scalar
        Adjusted stepsize 
    """

    # Parameters to the dual averaging algorithm.
    gamma = 0.1
    t0 = 10
    kappa = 0.75
    mu = math.log(1. * stepsize)

    # Initialize dual averaging algorithm.
    stepsizebar = stepsize
    
    # Save the history of attempted and averaged stepsizes, which
    # is useful mainly for checking the behavior of dual-averating.
    stepsize_seq = np.zeros(n_iter + 1)
    stepsizebar_seq = np.zeros(n_iter + 1)
    stepsize_seq[0] = stepsize
    stepsizebar_seq[0] = stepsizebar

    Hbar = 0
    for i in range(1, n_iter + 1):
        chain_state = mcmc_step(chain_state, stepsize)
        accept_prob = chain_state['accept_prob']
        w = 1 / (i + t0)
        Hbar = (1 - w) * Hbar + w * (accept_target - accept_prob)
        log_stepsize = mu - math.sqrt(i) / gamma * Hbar
        if log_stepsize < -100:
            stepsize = 0
        elif log_stepsize > 100:
            stepsize = float('inf')
        else:
            stepsize = math.exp(log_stepsize)
        eta = i ** -kappa
        stepsizebar = math.exp((1 - eta) * math.log(stepsizebar) + eta * math.log(stepsize))
        stepsize_seq[i] = stepsize
        stepsizebar_seq[i] = stepsizebar

    return chain_state, stepsizebar, stepsize_seq, stepsizebar_seq

In [ ]:
def RWMH(f, theta0, stepsize, n_warmup, n_samples, Sigma=None):
    """ 
    Wrapper function for tuning the proposal variance and sampling
    from the target using RWMH. 
    """
    
    n_param = len(np.atleast_1d(theta0))
    
    # Tune the proposal variance using dual-averaging.
    if n_warmup > 0:
        def mcmc_step(chain_state, stepsize):
            theta, logp, accepted, accept_prob = \
                RWMH_step(f, chain_state['theta'], chain_state['logp'], stepsize, Sigma)
            return {'theta': theta, 'logp': logp, 'accept_prob': accept_prob}
        init_state = {'theta': theta0, 'logp': f(theta0), 'accept_prob': 0}
        chain_state, stepsize, stepsize_seq, ave_stepsize_seq = \
            dual_average(mcmc_step, init_state, stepsize, accept_target=.234, n_iter=n_warmup)
        theta0 = chain_state['theta']
    else:
        stepsize_seq = None
        ave_stepsize_seq = None
        
    mcmc_samples = np.zeros((n_samples, n_param))
    accepted = np.zeros(n_samples)
    mcmc_samples[0,:] = theta0
    logp = f(theta0)
    for i in range(1, n_samples):
        mcmc_samples[i, :], logp, accepted[i], _ = \
            RWMH_step(f, mcmc_samples[i - 1, :], logp, stepsize, Sigma)
    accept_rate = np.mean(accepted[1:])
    
    return mcmc_samples, accept_rate, stepsize_seq, ave_stepsize_seq

In [ ]:
def Adap_RWMH(f, theta0, stepsize, n_warmup, n_cov_adap, n_samples):
    """ 
    Params
    ------
    stepsize: float
        Initial stepsize (proposal std)
    n_warmup: int
        Number of iterations for tuning the (isotropic) proposal variance
    n_cov_adap: int
        Number of iterations for estimating the covariance by running RWMH with 
        the tuned isotropic proposal variance.
    n_samples: int
        Number of sampling iteration during which the covariance will continue
        to be "diminishigly" adapted.
    """
    
    n_param = len(np.atleast_1d(theta0))
    
    # Tune the proposal variance using dual-averaging.
    if n_warmup > 0:
        def mcmc_step(chain_state, stepsize):
            theta, logp, accepted, accept_prob = \
                RWMH_step(f, chain_state['theta'], chain_state['logp'], stepsize)
            return {'theta': theta, 'logp': logp, 'accept_prob': accept_prob}
        init_state = {'theta': theta0, 'logp': f(theta0), 'accept_prob': 0}
        chain_state, stepsize, stepsize_seq, ave_stepsize_seq = \
            dual_average(mcmc_step, init_state, stepsize, accept_target=.234, n_iter=n_warmup)
        theta0 = chain_state['theta']
    
    if n_cov_adap > 0:
        cov_est_samples = np.zeros((n_samples, n_param))
        cov_est_samples[0,:] = theta0
        logp = f(theta0)
        for i in range(1, n_samples):
            cov_est_samples[i, :], logp, _, _ = \
                RWMH_step(f, cov_est_samples[i - 1, :], logp, stepsize)
        mu_hat = np.mean(cov_est_samples, 0)
        Sigma_hat = np.cov(cov_est_samples.T)
        stepsize = 2.38 / np.sqrt(n_param)
        
    epsilon = 10 ** -3 
    mcmc_samples = np.zeros((n_samples, n_param))
    accepted = np.zeros(n_samples)
    theta = theta0
    mcmc_samples[0,:] = theta
    logp = f(theta0)
    for i in range(1, n_samples):
        Sigma = (1 - epsilon) * stepsize * Sigma_hat + epsilon * stepsize * np.eye(n_param)
        theta, logp, accepted[i], _ = \
            RWMH_step(f, theta, logp, stepsize, Sigma)
        mu_hat, Sigma_hat = update_emp_cov(theta, mu_hat, Sigma_hat, i + n_cov_adap)
        mcmc_samples[i, :] = theta
    accept_rate = np.mean(accepted[1:])
    
    return mcmc_samples, accept_rate
                          
def update_emp_cov(theta, mu_hat, Sigma_hat, n):
    Sigma_hat = (n - 1) / n * Sigma_hat + \
        (n - 1) / n ** 2 * np.outer(theta - mu_hat, theta - mu_hat)
    mu_hat = (n - 1) / n * mu_hat + 1 / n * theta
    return mu_hat, Sigma_hat

In [ ]:
%%prun
def f_logp(theta):
    logp, _, _ = f(theta, req_grad=False)
    return logp

Sigma = 2.38 / np.sqrt(n_param) * np.cov(samples.T)
stepsize = 1

# Run MH with a fixed covariance.
n_samples = 2 * 10 ** 3
n_warmup = 0
tic = time.process_time() # Start clock
samples, accept_rate, stepsize_seq, ave_stepsize_seq = \
    RWMH(f_logp, theta0, stepsize, n_warmup, n_samples, Sigma)
toc = time.process_time()

In [ ]:
%%prun
def f_logp(theta):
    logp, _, _ = f(theta, req_grad=False)
    return logp

# Run adaptive MH to estimate the covariance.
n_warmup = 10 ** 4
n_cov_adap = 10 ** 4
n_adap_mcmc = 10 ** 3
seed = 1
np.random.seed(seed)
stepsize = 1 / math.sqrt(n_param)
samples, accept_rate = \
    Adap_RWMH(f_logp, theta0, stepsize, n_warmup, n_cov_adap, n_adap_mcmc)
Sigma = 2.38 / np.sqrt(n_param) * np.cov(samples.T)

# Run MH with a fixed covariance.
n_samples = 5 * 10 ** 3
tic = time.process_time() # Start clock
samples, accept_rate, stepsize_seq, ave_stepsize_seq = \
    RWMH(f_logp, theta0, stepsize, n_warmup, n_samples, Sigma)
toc = time.process_time()

In [ ]:
mcmc_output = {
    'samples': samples,
    'accept_rate': accept_rate,
    'n_warmup': n_warmup,
    'n_cov_adap': n_cov_adap,
    'n_adap_mcmc': n_adap_mcmc,
    'seed': seed,
    'theta0': theta0,
}
filename = 'jolly_seber_mh_output.npy'
with open(filename, 'wb') as file:
    pkl.dump(mcmc_output, file)

### Analyze the MCMC output.

In [ ]:
import os

def coda_ess(samples, normed=True):
    filenum = np.random.randint(2 ** 31) 
    # Append a random number to a file name to avoid conflicts.
    saveto = 'mchain{:d}.csv'.format(filenum)
    loadfrom = 'ess{:d}.csv'.format(filenum)
    np.savetxt(saveto, samples, delimiter=',')
    os.system(" ".join(["Rscript compute_ess.R", saveto, loadfrom]))
    ess = np.loadtxt(loadfrom, delimiter=',').copy()
    if normed:
        ess = ess / samples.shape[0]
    os.system(" ".join(["rm", saveto, loadfrom]))
    return ess

def mon_seq_ess(samples, normed=True):
    ess = [compute_ess(column, normed=True)[0] for column in samples.T]
    return np.squeeze(np.array(ess))

def batch_ess(x, n_batch=50, normed=True):
    batch_index = np.linspace(0, len(x), n_batch + 1).astype('int')
    batch_list = [x[batch_index[i]:batch_index[i+1]] for i in range(n_batch)]
    batch_mean = [np.mean(batch) for batch in batch_list]
    mcmc_var = len(x) / n_batch * np.var(batch_mean)
    ess = np.var(x) / mcmc_var
    if not normed: ess *= len(x)
    return ess

In [ ]:
filename = 'jolly_seber_dhmc_output.pkl'
with open(filename, 'rb') as file:
    mcmc_output = pkl.load(file)
samples = mcmc_output['samples']

In [ ]:
# Frequentist estimates.
phi_hat = np.array([.649, 1.015, .867, .564, .836, .790, .651, .985, .686, .884, .771, float('nan')])
phi_hat_sd = np.array([.114, .110, .107, .064, .075, .070, .056, .093, .080, .120, .128, float('nan')])
N_hat = np.array([float('nan'), 511.2, 779.1, 963.0, 945.3, 882.2, 802.5, 653.6, 628.8, 478.5, 506.4, 462.8, float('nan')])
N_hat_sd = np.array([float('nan'), 151.2, 129.3, 140.9, 125.5, 96.1, 74.8, 61.7, 61.9, 51.8, 65.8, 70.2, float('nan')])

In [ ]:
p_samples, phi_samples, U_samples, N_samples = \
    unpack_param(samples)

In [ ]:
np.vstack((np.mean(phi_samples, axis=0), phi_hat)).T

In [ ]:
np.vstack((np.std(phi_samples, axis=0), phi_hat_sd)).T

In [ ]:
np.vstack((np.mean(N_samples, 0), N_hat)).T 

In [ ]:
np.vstack((np.std(N_samples, 0), N_hat_sd)).T

In [ ]:
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(p_samples[:1000,:])
plt.subplot(1, 2, 2)
plt.hist(p_samples[:, 0], bins=25)
plt.show()

In [ ]:
plt.plot(U_samples[:1000,:])
plt.show()

### Investigate the correlation structure of the posterior

In [ ]:
param_index = np.hstack(tuple([index_U[i], index_p[i]] for i in range(T - 1)))
plt.imshow(np.corrcoef(samples[:, param_index].T), cmap='coolwarm')
plt.clim(-1, 1)
plt.colorbar()
plt.show()

In [ ]:
param_index = np.hstack(tuple([index_U[i], index_phi[i]] for i in range(T - 1)))
plt.imshow(np.corrcoef(samples[:, param_index].T), cmap='coolwarm')
plt.clim(-1, 1)
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(np.corrcoef(U_samples.T), cmap='coolwarm')
plt.xlabel('')
plt.clim(-1, 1)
plt.colorbar()
plt.show()

In [ ]:
# plt.hist2d(p_samples[:,0], np.log(U_samples[:,0]), bins=20, cmap='inferno')
plt.figure(figsize=(7,5), dpi=80)
plt.rcParams['font.size'] = 20
plt.hist2d(logodd(p_samples[:,0]), np.log10(U_samples[:,0]), bins=20, normed=True, cmap='inferno')
plt.xlabel(r'$\log(q_1 / (1 - q_1))$')
plt.ylabel(r'$\log_{10}(N_1)$')
plt.colorbar(ticks=[])
plt.tight_layout()
plt.savefig('jolly_seber_posterior_2dhist.pdf')
plt.show()

In [ ]:
plt.hist2d(np.log10(U_samples[:,2]), logit(phi_samples[:,0]), bins=20, cmap='inferno')
# plt.hist2d(logit(p_samples[:,0]), np.log(N_samples[:,0]), bins=20, cmap='inferno')
plt.colorbar()
plt.show()

In [ ]:
param_index = np.hstack(tuple([index_phi[i]] for i in range(T - 1)))
plt.imshow(np.corrcoef(samples[:, param_index].T), cmap='coolwarm')
plt.clim(-1, 1)
plt.colorbar()
plt.show()